# 30M Resistance/Support Detection

In [1]:
import pandas as pd
df = pd.read_csv("data/monthly_segments/CL_30m_2024-09.csv")
df2 = pd.read_csv("data/monthly_segments/CL_30m_2024-10.csv")
df = pd.concat([df,df2], ignore_index=True)
df.tail()

,datetime,open,high,low,close,volume
2070,2024-10-31 21:30:00,70.49,70.65,70.49,70.51,1564
2071,2024-10-31 22:00:00,70.51,70.54,70.43,70.54,1002
2072,2024-10-31 22:30:00,70.54,70.63,70.41,70.46,1058
2073,2024-10-31 23:00:00,70.45,70.52,70.35,70.37,894
2074,2024-10-31 23:30:00,70.37,70.52,70.32,70.51,1039


In [2]:
# Remove any NA rows that might be in data
df=df[df['volume']!=0]
df.reset_index(drop=True, inplace=True)
df.isna().sum()
df.tail()

,datetime,open,high,low,close,volume
2070,2024-10-31 21:30:00,70.49,70.65,70.49,70.51,1564
2071,2024-10-31 22:00:00,70.51,70.54,70.43,70.54,1002
2072,2024-10-31 22:30:00,70.54,70.63,70.41,70.46,1058
2073,2024-10-31 23:00:00,70.45,70.52,70.35,70.37,894
2074,2024-10-31 23:30:00,70.37,70.52,70.32,70.51,1039


# Support and Resitance FUNCTIONS

In [3]:
# Peek forward and back to check for consecutive lows and highs
def support(df1, l, peek_back, peek_forward): 
    for i in range(l-peek_back+1, l+1):
        if(df1.low[i]>df1.low[i-1]):
            return 0
    for i in range(l+1,l+peek_forward+1):
        if(df1.low[i]<df1.low[i-1]):
            return 0
    return 1

def resistance(df1, l, peek_back, peek_forward): 
    for i in range(l-peek_back+1, l+1):
        if(df1.high[i]<df1.high[i-1]):
            return 0
    for i in range(l+1,l+peek_forward+1):
        if(df1.high[i]>df1.high[i-1]):
            return 0
    return 1

In [4]:
# Make sure period looks good. 
import plotly.graph_objects as go
from datetime import datetime

fig = go.Figure(data=[go.Candlestick(x=df.index,
                open=df['open'],
                high=df['high'],
                low=df['low'],
                close=df['close'],
                increasing_line_color='green',
                decreasing_line_color='red'
                )])

fig.show()

In [5]:
sr = []
lookback=4
lookforward=2
for row in range(lookback, df.index.max() - lookforward): #len(df)-lookforward
    if support(df, row, lookback, lookforward):
        sr.append((row,df.low[row],1))
    if resistance(df, row, lookback, lookforward):
        sr.append((row,df.high[row],2))
print(sr)


[(4, np.float64(72.95), 1), (22, np.float64(73.91), 2), (37, np.float64(74.41), 2), (68, np.float64(72.06), 1), (69, np.float64(72.06), 1), (76, np.float64(70.38), 1), (86, np.float64(70.43), 2), (106, np.float64(70.11), 2), (109, np.float64(69.19), 1), (133, np.float64(68.82), 1), (134, np.float64(68.82), 1), (284, np.float64(69.08), 2), (323, np.float64(66.43), 2), (332, np.float64(65.91), 1), (358, np.float64(65.63), 1), (368, np.float64(67.05), 1), (390, np.float64(68.5), 2), (403, np.float64(67.81), 1), (406, np.float64(69.81), 2), (423, np.float64(69.24), 1), (425, np.float64(69.5), 2), (442, np.float64(69.92), 2), (459, np.float64(68.47), 1), (465, np.float64(69.39), 2), (471, np.float64(68.7), 1), (474, np.float64(69.0), 2), (475, np.float64(69.0), 2), (498, np.float64(70.7), 2), (530, np.float64(70.75), 2), (550, np.float64(71.92), 2), (560, np.float64(69.91), 1), (583, np.float64(68.58), 1), (594, np.float64(70.26), 2), (604, np.float64(68.95), 1), (648, np.float64(70.96), 1)

In [6]:
import plotly.graph_objects as go
from datetime import datetime
import matplotlib.pyplot as plt

dfpl = df.copy()
start = dfpl.index[0]
end = dfpl.index[-1]
levels_in_slice = [x for x in sr if start <= x[0] < end]

fig = go.Figure(data=[go.Candlestick(x=dfpl['datetime'],
                open=dfpl['open'],
                high=dfpl['high'],
                low=dfpl['low'],
                close=dfpl['close'])])

for row, yval, _ in levels_in_slice:
    fig.add_shape(type='line', x0=df['datetime'].min(), y0=yval,
                  x1=df['datetime'].max(),
                  y1=yval
                  )

fig.update_layout(
    showlegend=False,
    xaxis_rangeslider_visible=False,
    xaxis=dict(showgrid=False),
    yaxis=dict(showgrid=False),
    paper_bgcolor='black',
    plot_bgcolor='black',
    title=dict(text=f'Non-Collapsed Levels. Level Count: {str(len(levels_in_slice))}'),
    font=dict(color='white', size=20, family='Cascadia Black')
)

fig.show()


In [7]:
plotlist1 = [x[1] for x in sr if x[2]==1] # Supports
plotlist2 = [x[1] for x in sr if x[2]==2] # Resistances
initial_levels = len(sr)
collapse_thresh = 0.25
plotlist1.sort()
plotlist2.sort()

for i in range(1,len(plotlist1)):
    if(i>=len(plotlist1)):
        break
    if abs(plotlist1[i]-plotlist1[i-1]) <= collapse_thresh:
        # Keep more conservative level.
        if plotlist1[i] < plotlist1[i-1]:
            plotlist1.pop(i-1)
        else:
            plotlist1.pop(i)

for i in range(1,len(plotlist2)):
    if(i>=len(plotlist2)):
        break
    if abs(plotlist2[i]-plotlist2[i-1]) <= collapse_thresh:
        if plotlist2[i] > plotlist2[i-1]:
            plotlist2.pop(i-1)
        else:
            plotlist2.pop(i)

print(f"Collapsed {initial_levels-len(plotlist1)-len(plotlist2)} levels.")


Collapsed 45 levels.


In [8]:
rr = []
ss = []
for row in range(lookback, len(df) - lookforward): #len(df)-lookforward
    if support(df, row, lookback, lookforward):
        ss.append((row,df.low[row]))
    if resistance(df, row, lookback, lookforward):
        rr.append((row,df.high[row]))

In [9]:
import plotly.graph_objects as go
from datetime import datetime
import matplotlib.pyplot as plt

start = dfpl.index[0]
end = dfpl.index[-1]
ss_slice = [(r, y) for (r, y) in ss if start <= r < end]
rr_slice = [(r, y) for (r, y) in rr if start <= r < end]

fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['open'],
                high=dfpl['high'],
                low=dfpl['low'],
                close=dfpl['close'],
                increasing_line_color='green',
                decreasing_line_color='red')])

for row, yval in ss_slice:
    fig.add_shape(type='line', x0=start, y0=yval,
                  x1=end,
                  y1=yval,
                  line=dict(color="#8afffd",width=2),
                  name='Support',
                  opacity=0.75
                  )

for row, yval in rr_slice:
    fig.add_shape(type='line', x0=start, y0=yval,
                  x1=end,
                  y1=yval,
                  line=dict(color="#ff3c69",width=2),
                  name='Resistance',
                  opacity=0.75
                  )

fig.update_layout(
    showlegend=False,
    xaxis_rangeslider_visible=False,
    xaxis=dict(showgrid=False),
    yaxis=dict(showgrid=False),
    paper_bgcolor='black',
    plot_bgcolor='black',
    title=dict(text='Collapsed Support / Resistances'),
    font=dict(color='white', size=17, family='Cascadia Black')
)

fig.show()

In [10]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

# Time-decayed opacity vs last candle within slice, with faster decay in crowded zones
# Each level starts at its originating candle on the x-axis

dfpl = df[start:end]
df_idx = pd.to_datetime(dfpl["datetime"])
last_time = df_idx.max()
first_time = df_idx.min()
span_sec = max((last_time - first_time).total_seconds(), 1)

base_decay = 0.0       # isolated level: no decay
crowd_coeff = 0.25     # increase to speed up fade when crowded
window = 0.5           # price window for crowding (+/- window)

ss_slice = [(r, y) for (r, y) in ss if start <= r < end]
rr_slice = [(r, y) for (r, y) in rr if start <= r < end]
all_levels = [lvl for _, lvl in ss_slice + rr_slice]

# map from global row -> timestamp in slice space
row_to_time = {}
for i, global_row in enumerate(range(start, end)):
    row_to_time[global_row] = df_idx.iloc[i]


def time_decay_levels(levels):
    out = []
    for row, level in levels:
        level_time = row_to_time.get(row, last_time)
        age_sec = max((last_time - level_time).total_seconds(), 0)
        age_norm = age_sec / span_sec
        near_count = sum(1 for v in all_levels if abs(v - level) <= window)
        effective_decay = base_decay + crowd_coeff * max(0, near_count - 1)
        strength = float(np.exp(-effective_decay * age_norm))
        out.append((row, level, strength, level_time))
    return out

ss_strengths = time_decay_levels(ss_slice)
rr_strengths = time_decay_levels(rr_slice)

supports = pd.DataFrame(ss_strengths, columns=['row', 'level', 'strength', 'time'])
supports['type'] = 'support'
resistances = pd.DataFrame(rr_strengths, columns=['row', 'level', 'strength', 'time'])
resistances['type'] = 'resistance'

levels = pd.merge(supports, resistances, how='outer')
levels.to_csv("levels/30m_levels_24_09_10.csv")

fig = go.Figure(data=[go.Candlestick(
    x=df_idx,
    open=dfpl['open'], high=dfpl['high'], low=dfpl['low'], close=dfpl['close'],
    increasing_line_color='green', decreasing_line_color='red'
)])

x_end = df_idx.max()

for _, level, strength, t0 in ss_strengths:
    fig.add_shape(
        type='line', xref='x', yref='y',
        x0=t0, x1=x_end, y0=level, y1=level,
        line=dict(color='rgb(30,144,255)', width=2),
        opacity=strength,
    )

for _, level, strength, t0 in rr_strengths:
    fig.add_shape(
        type='line', xref='x', yref='y',
        x0=t0, x1=x_end, y0=level, y1=level,
        line=dict(color='rgb(220,20,60)', width=2),
        opacity=strength,
    )

fig.update_layout(
    showlegend=False,
    xaxis_rangeslider_visible=False,
    xaxis=dict(showgrid=False),
    yaxis=dict(showgrid=False),
    paper_bgcolor='black',
    plot_bgcolor='black',
    title=dict(text=f'Levels, Time and Crowding Decayed. 30 minutes. {first_time.date()} to {last_time.date()}',
    font=dict(color='white', size=25, family='Cascadia Black'))
)

fig.show()


In [11]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

# 1H levels on last 30 days, with row-aware collapse and crowd-aware decay

_df = df.copy()
_df['datetime'] = pd.to_datetime(_df['datetime'])
_df = _df.set_index('datetime').sort_index()
df_1h = _df.resample('1h').agg({'open':'first','high':'max','low':'min','close':'last','volume':'sum'}).dropna().reset_index()

earliest = df_1h['datetime'].min()
cutoff = earliest + pd.Timedelta(days=30)
df_h = df_1h[df_1h['datetime'] <= cutoff].reset_index(drop=True)

n1, n2 = 4, 2
levels_h = []
for row in range(n1, len(df_h) - n2):
    if support(df_h, row, n1, n2):
        levels_h.append((row, df_h.low[row], 1))
    if resistance(df_h, row, n1, n2):
        levels_h.append((row, df_h.high[row], 2))

# collapse while keeping row of first occurrence
def collapse_pairs(pairs, thresh=0.5):
    pairs = sorted(pairs, key=lambda x: x[1])
    kept = []
    for row, price in pairs:
        if not kept or abs(price - kept[-1][1]) > thresh:
            kept.append((row, price))
    return kept

ss_h = collapse_pairs([(r, y) for r, y, t in levels_h if t == 1])
rr_h = collapse_pairs([(r, y) for r, y, t in levels_h if t == 2])

# time decay with crowding; lines start at originating candle
last_time = df_h['datetime'].max()
first_time = df_h['datetime'].min()
span_sec = max((last_time - first_time).total_seconds(), 1)
all_levels = [lvl for _, lvl in ss_h + rr_h]
row_to_time = {i: df_h['datetime'].iloc[i] for i in range(len(df_h))}

def time_decay_levels(levels):
    out = []
    for row, level in levels:
        level_time = row_to_time.get(row, last_time)
        age_sec = max((last_time - level_time).total_seconds(), 0)
        age_norm = age_sec / span_sec
        near_count = sum(1 for v in all_levels if abs(v - level) <= window)
        effective_decay = base_decay + crowd_coeff * max(0, near_count - 1)
        strength = float(np.exp(-effective_decay * age_norm))
        out.append((row, level, strength, level_time))
    return out

ss_strengths = time_decay_levels(ss_h)
rr_strengths = time_decay_levels(rr_h)

supports = pd.DataFrame(ss_strengths, columns=['row', 'level', 'strength', 'time'])
supports['type'] = 'support'
resistances = pd.DataFrame(rr_strengths, columns=['row', 'level', 'strength', 'time'])
resistances['type'] = 'resistance'

levels = pd.merge(supports, resistances, how='outer')
levels.to_csv("levels/1h_levels_24_09_10.csv")

fig = go.Figure(data=[go.Candlestick(
    x=df_h['datetime'],
    open=df_h['open'], high=df_h['high'], low=df_h['low'], close=df_h['close'],
    increasing_line_color='green', decreasing_line_color='red'
)])

x_end = df_h['datetime'].max()

for _, level, strength, t0 in ss_strengths:
    fig.add_shape(
        type='line', xref='x', yref='y',
        x0=t0, x1=x_end, y0=level, y1=level,
        line=dict(color='rgb(30,144,255)', width=2),
        opacity=strength,
    )

for _, level, strength, t0 in rr_strengths:
    fig.add_shape(
        type='line', xref='x', yref='y',
        x0=t0, x1=x_end, y0=level, y1=level,
        line=dict(color='rgb(220,20,60)', width=2),
        opacity=strength,
    )


fig.update_layout(
    showlegend=False,
    xaxis_rangeslider_visible=False,
    xaxis=dict(showgrid=False),
    yaxis=dict(showgrid=False),
    paper_bgcolor='black',
    plot_bgcolor='black',
    title=dict(text=f'Levels, Time and Crowding Decayed. 1 Hour. {first_time.date()} to {last_time.date()}',
    font=dict(color='white', size=25, family='Cascadia Black'))
)

fig.show()


In [12]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

window = 1.5         # price window for crowding (+/- window)

# 4H levels on last month of data (standalone pipeline)
# Uses existing support/resistance functions; does not alter global start/end.

# Prepare 4h bars
_df = df.copy()
_df['datetime'] = pd.to_datetime(_df['datetime'])
_df = _df.set_index('datetime').sort_index()
df_4h = _df.resample('4h').agg({'open':'first','high':'max','low':'min','close':'last','volume':'sum'}).dropna().reset_index()

# Last month slice
earliest = df_4h['datetime'].min()
cutoff = earliest + pd.Timedelta(days=30)
df_h = df_4h[df_4h['datetime'] <= cutoff].reset_index(drop=True)
start_h, end_h = 0, len(df_h)

# Run support/resistance on 4h slice
sr = []
n1, n2 = 4, 2
for row in range(n1, len(df_h) - n2):
    if support(df_h, row, n1, n2):
        sr.append((row, df_h.low[row], 1))
    if resistance(df_h, row, n1, n2):
        sr.append((row, df_h.high[row], 2))

# Collapse nearby levels
plotlist1_4h = [x[1] for x in sr if x[2] == 1]
plotlist2_4h = [x[1] for x in sr if x[2] == 2]
plotlist1_4h.sort(); plotlist2_4h.sort()

for i in range(1,len(plotlist1_4h)):
    if(i>=len(plotlist1_4h)):
        break
    if abs(plotlist1_4h[i]-plotlist1_4h[i-1]) <= collapse_thresh: # Reuses same collapse Thresh
        # Keep more conservative level.
        if plotlist1_4h[i] < plotlist1_4h[i-1]:
            plotlist1_4h.pop(i-1)
        else:
            plotlist1_4h.pop(i)

for i in range(1,len(plotlist2_4h)):
    if(i>=len(plotlist2_4h)):
        break
    if abs(plotlist2_4h[i]-plotlist2_4h[i-1]) <= collapse_thresh:
        if plotlist2_4h[i] > plotlist2_4h[i-1]:
            plotlist2_4h.pop(i-1)
        else:
            plotlist2_4h.pop(i)

# Separate supports/resistances (post-collapse, keep original rows)
ss = [(r, y) for r, y, t in sr if t == 1]
rr = [(r, y) for r, y, t in sr if t == 2]

# Time-decayed opacity with crowding; each line starts at its level candle
last_time = df_h['datetime'].max()
first_time = df_h['datetime'].min()
span_sec = max((last_time - first_time).total_seconds(), 1)
all_levels = [lvl for _, lvl in ss + rr]
row_to_time = {i: df_h['datetime'].iloc[i] for i in range(len(df_h))}

ss_strengths = time_decay_levels(ss)
rr_strengths = time_decay_levels(rr)

supports = pd.DataFrame(ss_strengths, columns=['row', 'level', 'strength', 'time'])
supports['type'] = 'support'
resistances = pd.DataFrame(rr_strengths, columns=['row', 'level', 'strength', 'time'])
resistances['type'] = 'resistance'

levels = pd.merge(supports, resistances, how='outer')
levels.to_csv("levels/4h_levels_24_09_10.csv")

fig = go.Figure(data=[go.Candlestick(
    x=df_h['datetime'],
    open=df_h['open'], high=df_h['high'], low=df_h['low'], close=df_h['close'],
    increasing_line_color='green', decreasing_line_color='red'
)])

x_end = df_h['datetime'].max()

for _, level, strength, t0 in ss_strengths:
    fig.add_shape(
        type='line', xref='x', yref='y',
        x0=t0, x1=x_end, y0=level, y1=level,
        line=dict(color='rgb(30,144,255)', width=2),
        opacity=strength,
    )

for _, level, strength, t0 in rr_strengths:
    fig.add_shape(
        type='line', xref='x', yref='y',
        x0=t0, x1=x_end, y0=level, y1=level,
        line=dict(color='rgb(220,20,60)', width=2),
        opacity=strength,
    )

fig.update_layout(
    showlegend=False,
    xaxis_rangeslider_visible=False,
    xaxis=dict(showgrid=False),
    yaxis=dict(showgrid=False),
    paper_bgcolor='black',
    plot_bgcolor='black',
    title=dict(text=f'Levels, Time and Crowding Decayed. 4 Hour. {first_time.date()} to {last_time.date()}',
    font=dict(color='white', size=25, family='Cascadia Black'))
)

fig.show()


In [13]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

# 1D levels on last 60 days, with row-aware collapse and crowd-aware decay

_df = df.copy()
_df['datetime'] = pd.to_datetime(_df['datetime'])
_df = _df.set_index('datetime').sort_index()
df_tf = _df.resample('1D').agg({'open':'first','high':'max','low':'min','close':'last','volume':'sum'}).dropna().reset_index()

earliest = df_tf['datetime'].min()
cutoff = earliest + pd.Timedelta(days=60)
df_t = df_tf[df_tf['datetime'] <= cutoff].reset_index(drop=True)

n1, n2 = 3, 2
levels_t = []
for row in range(n1, len(df_t) - n2):
    if support(df_t, row, n1, n2):
        levels_t.append((row, df_t.low[row], 1))
    if resistance(df_t, row, n1, n2):
        levels_t.append((row, df_t.high[row], 2))

# collapse while keeping row of first occurrence
def collapse_pairs(pairs, thresh=0.5):
    pairs = sorted(pairs, key=lambda x: x[1])
    kept = []
    for row, price in pairs:
        if not kept or abs(price - kept[-1][1]) > thresh:
            kept.append((row, price))
    return kept

ss_t = collapse_pairs([(r, y) for r, y, t in levels_t if t == 1])
rr_t = collapse_pairs([(r, y) for r, y, t in levels_t if t == 2])

# time decay with crowding; lines start at originating candle
last_time = df_t['datetime'].max()
first_time = df_t['datetime'].min()
span_sec = max((last_time - first_time).total_seconds(), 1)
base_decay = 0.0
crowd_coeff = 0.7
window = 0.5
all_levels = [lvl for _, lvl in ss_t + rr_t]
row_to_time = {i: df_t['datetime'].iloc[i] for i in range(len(df_t))}

def time_decay_levels(levels):
    out = []
    for row, level in levels:
        level_time = row_to_time.get(row, last_time)
        age_sec = max((last_time - level_time).total_seconds(), 0)
        age_norm = age_sec / span_sec
        near_count = sum(1 for v in all_levels if abs(v - level) <= window)
        effective_decay = base_decay + crowd_coeff * max(0, near_count - 1)
        strength = float(np.exp(-effective_decay * age_norm))
        out.append((row, level, strength, level_time))
    return out

ss_strengths = time_decay_levels(ss_t)
rr_strengths = time_decay_levels(rr_t)

supports = pd.DataFrame(ss_strengths, columns=['row', 'level', 'strength', 'time'])
supports['type'] = 'support'
resistances = pd.DataFrame(rr_strengths, columns=['row', 'level', 'strength', 'time'])
resistances['type'] = 'resistance'

levels = pd.merge(supports, resistances, how='outer')
levels.to_csv("levels/1d_levels_24_09_10.csv")

fig = go.Figure(data=[go.Candlestick(
    x=df_t['datetime'],
    open=df_t['open'], high=df_t['high'], low=df_t['low'], close=df_t['close'],
    increasing_line_color='green', decreasing_line_color='red'
)])

x_end = df_t['datetime'].max()

for _, level, strength, t0 in ss_strengths:
    fig.add_shape(
        type='line', xref='x', yref='y',
        x0=t0, x1=x_end, y0=level, y1=level,
        line=dict(color='rgb(30,144,255)', width=2),
        opacity=strength,
    )

for _, level, strength, t0 in rr_strengths:
    fig.add_shape(
        type='line', xref='x', yref='y',
        x0=t0, x1=x_end, y0=level, y1=level,
        line=dict(color='rgb(220,20,60)', width=2),
        opacity=strength,
    )

fig.update_layout(
    showlegend=False,
    xaxis_rangeslider_visible=False,
    xaxis=dict(showgrid=False),
    yaxis=dict(showgrid=False),
    paper_bgcolor='black',
    plot_bgcolor='black',
    title='1D Levels (Last 60 Days): Crowd-aware Decay from Origin Candle',
    font=dict(color='white', size=15, family='Cascadia Black')
)

fig.show()
